In [37]:
import requests, zipfile, io, sys
import pandas as pd

# Data Collection & EDA

__a. Lines Information__

In [38]:
header = {'api_key': '29f3d28264f8450f96506480de622a24'}
response = requests.get("https://api.wmata.com/Rail.svc/json/jLines", headers=header) 
response.status_code

200

In [39]:
lines_json = response.json()
lines_arr = lines_json["Lines"]
lines = pd.DataFrame.from_records(lines_arr)
lines

,LineCode,DisplayName,StartStationCode,EndStationCode,InternalDestination1,InternalDestination2
0,BL,Blue,J03,G05,,
1,GR,Green,F11,E10,,
2,OR,Orange,K08,D13,,
3,RD,Red,A15,B11,A11,B08
4,SV,Silver,N06,G05,,
5,YL,Yellow,C15,E06,E01,


__b. Stations Information__

In [40]:
r = requests.get("https://api.wmata.com/Rail.svc/json/jStations", headers=header) 
r.status_code

200

In [41]:
stations_json = r.json()
stations_arr = stations_json["Stations"]
stations = pd.DataFrame.from_records(stations_arr)
stations.to_csv('out.csv')

The address data is nested in a dict. Let's give each field in the dict a new column.

In [42]:
addresses = stations['Address'].apply(pd.Series)
stations = pd.concat([stations.drop(['Address'], axis=1), addresses], axis=1)
stations[:1]

,Code,Name,StationTogether1,StationTogether2,LineCode1,LineCode2,LineCode3,LineCode4,Lat,Lon,Street,City,State,Zip
0,A01,Metro Center,C01,,RD,None,None,None,38.898303,-77.028099,607 13th St. NW,Washington,DC,20005


In [43]:
stations[stations['StationTogether1'] != '']

,Code,Name,StationTogether1,StationTogether2,LineCode1,LineCode2,LineCode3,LineCode4,Lat,Lon,Street,City,State,Zip
0,A01,Metro Center,C01,,RD,None,None,None,38.898303,-77.028099,607 13th St. NW,Washington,DC,20005
15,B01,Gallery Pl-Chinatown,F01,,RD,None,None,None,38.898340,-77.021851,630 H St. NW,Washington,DC,20001
20,B06,Fort Totten,E06,,RD,None,None,None,38.951777,-77.002174,550 Galloway St NE,Washington,DC,20011
27,C01,Metro Center,A01,,BL,OR,SV,None,38.898303,-77.028099,607 13th St. NW,Washington,DC,20005
44,D03,L'Enfant Plaza,F03,,BL,OR,SV,None,38.884775,-77.021964,600 Maryland Avenue SW,Washington,DC,20024
60,E06,Fort Totten,B06,,GR,YL,None,None,38.951777,-77.002174,550 Galloway St NE,Washington,DC,20011
65,F01,Gallery Pl-Chinatown,B01,,GR,YL,None,None,38.898340,-77.021851,630 H St. NW,Washington,DC,20001
67,F03,L'Enfant Plaza,D03,,GR,YL,None,None,38.884775,-77.021964,600 Maryland Avenue SW,Washington,DC,20024


In [44]:
stations[stations['StationTogether1'] != ''].shape

(8, 14)

There are 8 stations with more than one entrance.

In [45]:
stations.shape

(102, 14)

Note that there are 102 metro stations respresented in our dataset.

__c. Station to Station__

In [46]:
r = requests.get("https://api.wmata.com/Rail.svc/json/jSrcStationToDstStationInfo", headers=header) 
r.status_code

200

In [47]:
station_to_station_json = r.json()
station_to_station_arr = station_to_station_json["StationToStationInfos"]
station_to_station = pd.DataFrame.from_records(station_to_station_arr)
station_to_station.head()

,SourceStation,DestinationStation,CompositeMiles,RailTime,RailFare
0,A01,A02,0.75,2,"{'PeakTime': 2.0, 'OffPeakTime': 2.0, 'SeniorD..."
1,A01,A03,1.23,4,"{'PeakTime': 2.25, 'OffPeakTime': 2.25, 'Senio..."
2,A01,A04,2.40,6,"{'PeakTime': 2.25, 'OffPeakTime': 2.25, 'Senio..."
3,A01,A05,3.10,9,"{'PeakTime': 2.3, 'OffPeakTime': 2.3, 'SeniorD..."
4,A01,A06,3.73,11,"{'PeakTime': 2.6, 'OffPeakTime': 2.5, 'SeniorD..."


The fare data is nested in a dict with three fields: _OffPeakTime_, _PeakTime_, and _SeniorDisabled_. Let's put them in their own columns.

In [48]:
fares = station_to_station['RailFare'].apply(pd.Series)
station_to_station = pd.concat([station_to_station.drop(['RailFare'], axis=1), fares], axis=1)
station_to_station.head()

,SourceStation,DestinationStation,CompositeMiles,RailTime,PeakTime,OffPeakTime,SeniorDisabled
0,A01,A02,0.75,2,2.00,2.00,1.10
1,A01,A03,1.23,4,2.25,2.25,1.10
2,A01,A04,2.40,6,2.25,2.25,1.10
3,A01,A05,3.10,9,2.30,2.30,1.15
4,A01,A06,3.73,11,2.60,2.50,1.25


Note that there are 10,300 rows for station to station info. The station to station info should have a row for each combination of stations, including both directions separately (routes __A -> B__ _and_ __B -> A__ should both be counted because their fares/times may differ). The number of combinations of 2 stations out of 102 total is calculated with the combinations formula (nCr), and comes out to 5151, which is half of 10,300 (minus 2 missing rows for combinations of the same station (A -> A)). 

So 10,300 is the number of possible combinations of stations in the system, including _some_ of the combinations of a station and itself. Note that there is a real fare for "routes" from a station to itself, meaning that users are actually charged for entering and then exiting the same station. In 2016, WMATA implemented a grace period that would refund users if they leave the station in 15 minutes.

__d. GTFS Static__

The GTFS (General Transit Feed Specification) is an open standard data format used by WMATA to store its more in-depth transit data. We can retrieve data such as real-time ridership, train locations, speeds, and service alerts, which is the stuff we'll need to do some real data science.

First I'm trying to get the static (non real-time) version of the data. It is updated daily, so still plenty useful for analysis

In [49]:
response = requests.get("https://api.wmata.com/gtfs/rail-gtfs-static.zip", headers=header) 
response.status_code

200

In [50]:
z = zipfile.ZipFile(io.BytesIO(response.content))
z.extractall(path='./gtfs_static') # extract the data to './gtfs_static'

In [51]:
trips = pd.read_csv('./gtfs_static/trips.txt')
stops = pd.read_csv('./gtfs_static/stops.txt')

__e. GTFS Real Time__

In [52]:
import google.protobuf
import gtfs_realtime_pb2 as rt

This is a protocol format file we need to set up our system to be able to read the data from the protocol buffer file. I'll compile the protocol buffer into the CWD

In [53]:
response = requests.get("https://gtfs.org/realtime/gtfs-realtime.proto") 
!protoc -I=. --python_out=. ./gtfs-realtime.proto

__Get Vehicle Positions__ - the real-time lattitude, longitude, bearing, occupancy status and more for each train in the system

In [54]:
response = requests.get("https://api.wmata.com/gtfs/rail-gtfsrt-vehiclepositions.pb", headers=header) 
positions_pb = rt.FeedMessage()
positions_pb.ParseFromString(response.content)
positions_pb

header {
  gtfs_realtime_version: "2.0"
  incrementality: FULL_DATASET
  timestamp: 1721322043
}
entity {
  id: "0"
  is_deleted: false
  vehicle {
    trip {
      trip_id: "5952135_19920"
      start_time: "13:03:00"
      start_date: "20240718"
      schedule_relationship: SCHEDULED
      route_id: "RED"
      direction_id: 0
    }
    position {
      latitude: 39.1209068
      longitude: -77.1653671
      bearing: 145
    }
    current_stop_sequence: 1
    current_status: STOPPED_AT
    timestamp: 1721322043
    stop_id: "PF_A15_C"
    vehicle {
      id: "260"
      label: "113"
      license_plate: "8_3229-3228.3278-3279.3136-3137.3218-3219"
    }
    occupancy_status: EMPTY
  }
}
entity {
  id: "1"
  is_deleted: false
  vehicle {
    trip {
      trip_id: "5952229_19920"
      start_time: "12:33:00"
      start_date: "20240718"
      schedule_relationship: SCHEDULED
      route_id: "RED"
      direction_id: 0
    }
    position {
      latitude: 38.9422798
      longitude: -77.

__Get Trip Updates__ - any fluctuation in the timetable (delays, cancellations, changed routes)

In [55]:
response = requests.get("https://api.wmata.com/gtfs/rail-gtfsrt-tripupdates.pb", headers=header) 
updates_pb = rt.FeedMessage()
updates_pb.ParseFromString(response.content)
updates_pb

header {
  gtfs_realtime_version: "2.0"
  incrementality: FULL_DATASET
  timestamp: 1721322045
}
entity {
  id: "0"
  trip_update {
    trip {
      trip_id: "5952135_19920"
      start_time: "13:03:00"
      start_date: "20240718"
      schedule_relationship: SCHEDULED
      route_id: "RED"
      direction_id: 0
    }
    stop_time_update {
      stop_sequence: 1
      departure {
        time: 1721322180
        uncertainty: 0
      }
      stop_id: "PF_A15_C"
      schedule_relationship: SCHEDULED
    }
    stop_time_update {
      stop_sequence: 2
      arrival {
        time: 1721322372
        uncertainty: 0
      }
      stop_id: "PF_A14_C"
      schedule_relationship: SCHEDULED
    }
    stop_time_update {
      stop_sequence: 3
      arrival {
        time: 1721322566
        uncertainty: 0
      }
      stop_id: "PF_A13_C"
      schedule_relationship: SCHEDULED
    }
    stop_time_update {
      stop_sequence: 4
      arrival {
        time: 1721322712
        uncertainty: 0


__Get Service Alerts__ - stop moved, unforeseen events affecting a station, route or the entire network

In [56]:
response = requests.get("https://api.wmata.com/gtfs/rail-gtfsrt-alerts.pb", headers=header) 
alerts_pb = rt.FeedMessage()
alerts_pb.ParseFromString(response.content)
alerts_pb

header {
  gtfs_realtime_version: "2.0"
  incrementality: FULL_DATASET
  timestamp: 1721322039
}
entity {
  id: "8f43ec6d-fa1f-ef11-840a-0022481ee669"
  alert {
    informed_entity {
      route_id: "RED"
    }
    cause: MAINTENANCE
    effect: SIGNIFICANT_DELAYS
    url {
      translation {
        text: ""
        language: "en-us"
      }
    }
    header_text {
      translation {
        text: "Shuttle buses replace trains between Glenmont & Takoma for summer construction work through August 31st. Info: wmata.com"
        language: "en-us"
      }
    }
    description_text {
      translation {
        text: "Shuttle buses replace trains between Glenmont & Takoma for summer construction work through August 31st. Info: wmata.com"
        language: "en-us"
      }
    }
  }
}
entity {
  id: "e2df0838-fb1f-ef11-840a-0022481ee669"
  alert {
    informed_entity {
      route_id: "RED"
    }
    cause: MAINTENANCE
    effect: SIGNIFICANT_DELAYS
    url {
      translation {
        t

__Real-Time Rail Predictions__ - WMATA also provides realtime predictions in the API, which is probably what they use for their next train screens at the stations. This data is updated every 20-30s.

In [57]:
response = requests.get("https://api.wmata.com/StationPrediction.svc/json/GetPrediction/All", headers=header) 
rt_predictions_json = response.json()
rt_predictions_arr = rt_predictions_json["Trains"]
rt_predictions = pd.DataFrame.from_records(rt_predictions_arr)
rt_predictions

,Car,Destination,DestinationCode,DestinationName,Group,Line,LocationCode,LocationName,Min
0,6,Shady Grove,None,Shady Grove,2,RD,A01,Metro Center,BRD
1,8,Takoma,B07,Takoma,1,RD,A06,Van Ness-UDC,BRD
2,8,Takoma,B07,Takoma,1,RD,A08,Friendship Heights,BRD
3,6,Shady Grv,None,Shady Grv,2,RD,A09,Bethesda,BRD
4,8,Takoma,B07,Takoma,1,RD,A10,Medical Center,BRD
...,...,...,...,...,...,...,...,...,...
537,6,Dulles,N10,Washington Dulles International Airport,2,SV,N09,Innovation Center,33
538,-,Largo,G05,Downtown Largo,1,SV,N09,Innovation Center,35
539,None,ssenger,None,No Passenger,2,No,B08,Silver Spring,
540,None,ssenger,None,No Passenger,2,No,B09,Forest Glen,---


# What Affects WMATA's Predicted Arrival Times?

This will be a test of coordinating real-time data from multiple http requests. The goal is to pull all of the relevant data at the same time (approx. 1:30 pm on 7/9/2024) (each request may take a few seconds, but for our purposes we'll consider this simultaneous), collect it into a dataframe, and visualize the relationships between the various independent variables and the WMATA system's predicted arrival time for a given train. We're kind of doing data science _on_ data science, since we're analyzing the effects of variables on the results of a predictive model.

- Goal 1: plot arrival predictions vs nearest train distance

### Effect of Nearest Train Distance on Predicted Arrival Times

How close is the nearest train coming our way on X line? And how does this distance affect the arrival time predicted by DC's metro system? The answer may be obvious, but the relationship may not be as straight-cut or linear as we assume, so let's make some plots.

In [58]:
# collect vehicle position data into a dataframe
positions_arr = []
for entity in positions_pb.entity:
    if entity.HasField('vehicle'):
        positions_arr.append([entity.vehicle.position.latitude, entity.vehicle.position.longitude, entity.vehicle.position.bearing, 
                              entity.vehicle.vehicle.id, entity.vehicle.trip.trip_id, entity.vehicle.trip.route_id, entity.vehicle.trip.direction_id, entity.vehicle.stop_id])
        
rt_positions = pd.DataFrame(positions_arr, columns=['latitude', 'longitude', 'bearing', 'vehicle_id', 'trip_id', 'route_id', 'direction_id', 'stop_id'])
rt_positions

,latitude,longitude,bearing,vehicle_id,trip_id,route_id,direction_id,stop_id
0,39.120907,-77.165367,145.0,260,5952135_19920,RED,0,PF_A15_C
1,38.942280,-77.062485,156.0,259,5952229_19920,RED,0,PF_A06_C
2,39.084236,-77.146248,138.0,254,5952230_19920,RED,0,PF_A14_C
3,39.047127,-77.112465,159.0,290,5952054_19920,RED,0,PF_A12_C
4,38.997330,-77.096725,175.0,280,5951951_19920,RED,0,PF_A10_C
...,...,...,...,...,...,...,...,...
86,38.923763,-77.211945,240.0,453,6216874_19920,SILVER,1,PF_N01_C
87,38.947884,-77.340286,293.0,251,6216784_19920,SILVER,1,PF_N06_C
88,38.961632,-77.452263,180.0,350,6216869_19920,SILVER,1,PF_N10_C
89,38.948067,-76.871925,42.0,422,6217959_19920,ORANGE,0,PF_D13_C


Note that there are around 90 trains active in the system. This number fluctuates when we re-request the data because it is updated in real time. 

The difficulty lies in connecting each arrival time prediction to the the nearest train in the same line, on the same track, going the right direction. Each prediction has a station code, line, and destination code. The trips.txt file links destinations to train_ids and trip_ids, so we could get all the train_ids that correspond to a given predictions' destination. The problem is, this will also get trains that are already past our station, which should not be included in our calculation of the nearest train.

To illustrate this issue, __say we're at Fort Totten (station code E06) and trying to take the green line south__ (to L'Enfant, or wherever). We need our train to not just be going the right direction, it must also not already be past Fort Totten. To filter to only those trains that we can catch, we need each train's stop_id, telling us where it is in the line.

<center><img src="dc_metro_map_green.png"/></center>

In [59]:
# Get all trips on a line going the same direction
going_to_branch_ave = trips[(trips['route_id'] == 'GREEN') & (trips['direction_id'] == 1)]
# get all active gree line trains to BRANCH AVE 
trips_to_branch_ave = rt_positions[rt_positions['trip_id'].isin(going_to_branch_ave['trip_id'])]
trips_to_branch_ave

,latitude,longitude,bearing,vehicle_id,trip_id,route_id,direction_id,stop_id
69,39.010387,-76.911919,215.0,255,6217374_19920,GREEN,1,PF_E10_C
70,38.952370,-76.985947,276.0,325,6216856_19920,GREEN,1,PF_E07_2
71,38.985245,-76.925491,199.0,342,6216840_19920,GREEN,1,PF_E10_C
72,38.931602,-77.028877,246.0,364,6217373_19920,GREEN,1,PF_E05_C
75,38.893227,-77.021973,180.0,301,6216855_19920,GREEN,1,PF_F02_C
76,38.863274,-76.995628,167.0,450,6216851_19920,GREEN,1,PF_F06_C
77,38.851070,-76.956589,80.0,292,6217372_19920,GREEN,1,PF_F09_C
78,38.844635,-76.932915,139.0,272,6216853_19920,GREEN,1,PF_F10_C
90,38.826759,-76.911781,129.0,370,6216854_19920,GREEN,1,PF_F11_C


These are all the active trains going south on the green line. But if we're at Fort Totten, we can't get to the trains that are already south of us. We need to know the current station_id of each train to filter these out. I wrote a function in util.py that determines if a train is catchable to us base on our station id, its station id, and its direction.

In [60]:
# Each stop has a parent station, unless it is a parent station, then the field is filled with 'NaN' (which is type float)
# fill in these values with the station's own ID so we can use parent_id for every stop
for index, row in stops.iterrows():
    if type(row['parent_station']) == float:
        stops.at[index, 'parent_station'] = stops.at[index, 'stop_id']

# THIS WILL NEED TO BE REVISED, IT IS HIDEOUS
# Add station_id to branch ave trips(trains), this gives the current location of the train
trips_to_branch_ave['station_id'] = pd.Series()
for index, row in trips_to_branch_ave.iterrows():
    parent_station = stops['parent_station'][stops['stop_id'] == trips_to_branch_ave.at[index, 'stop_id']].iloc[0][4:]
    if len(parent_station) > 3: 
         trips_to_branch_ave.at[index, 'station_id']
    trips_to_branch_ave.at[index, 'station_id'] = stops['parent_station'][stops['stop_id'] == trips_to_branch_ave.at[index, 'stop_id']].iloc[0][4:]

/tmp/ipykernel_75/2774635542.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trips_to_branch_ave['station_id'] = pd.Series()


In [61]:
trips_to_branch_ave

,latitude,longitude,bearing,vehicle_id,trip_id,route_id,direction_id,stop_id,station_id
69,39.010387,-76.911919,215.0,255,6217374_19920,GREEN,1,PF_E10_C,E10
70,38.952370,-76.985947,276.0,325,6216856_19920,GREEN,1,PF_E07_2,E07
71,38.985245,-76.925491,199.0,342,6216840_19920,GREEN,1,PF_E10_C,E10
72,38.931602,-77.028877,246.0,364,6217373_19920,GREEN,1,PF_E05_C,E05
75,38.893227,-77.021973,180.0,301,6216855_19920,GREEN,1,PF_F02_C,F02
76,38.863274,-76.995628,167.0,450,6216851_19920,GREEN,1,PF_F06_C,F06
77,38.851070,-76.956589,80.0,292,6217372_19920,GREEN,1,PF_F09_C,F09
78,38.844635,-76.932915,139.0,272,6216853_19920,GREEN,1,PF_F10_C,F10
90,38.826759,-76.911781,129.0,370,6216854_19920,GREEN,1,PF_F11_C,F11


In [62]:
import util;

In [63]:
trips_to_branch_ave['catchable'] = pd.Series()
for index, row in trips_to_branch_ave.iterrows():
    trips_to_branch_ave.at[index, 'catchable'] = util.is_catchable("E06", "GR", 1, row['station_id'])

/tmp/ipykernel_75/2191244077.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trips_to_branch_ave['catchable'] = pd.Series()


In [64]:
trips_to_branch_ave

,latitude,longitude,bearing,vehicle_id,trip_id,route_id,direction_id,stop_id,station_id,catchable
69,39.010387,-76.911919,215.0,255,6217374_19920,GREEN,1,PF_E10_C,E10,False
70,38.952370,-76.985947,276.0,325,6216856_19920,GREEN,1,PF_E07_2,E07,False
71,38.985245,-76.925491,199.0,342,6216840_19920,GREEN,1,PF_E10_C,E10,False
72,38.931602,-77.028877,246.0,364,6217373_19920,GREEN,1,PF_E05_C,E05,False
75,38.893227,-77.021973,180.0,301,6216855_19920,GREEN,1,PF_F02_C,F02,False
76,38.863274,-76.995628,167.0,450,6216851_19920,GREEN,1,PF_F06_C,F06,False
77,38.851070,-76.956589,80.0,292,6217372_19920,GREEN,1,PF_F09_C,F09,False
78,38.844635,-76.932915,139.0,272,6216853_19920,GREEN,1,PF_F10_C,F10,False
90,38.826759,-76.911781,129.0,370,6216854_19920,GREEN,1,PF_F11_C,F11,False


In [65]:
our_station_id = "B08" #(Silver Spring)
line = 'RD'
direction = 0 # to glenmont
train_station_id = "A13" #(glenmont)

util.is_catchable(our_station_id, line, direction, train_station_id)

False

#### Prediction destination -> line & direction

### AVG minimum wait time at a station for each hour in a day